# L3: Supervised Fine-Tuning (SFT) 監督式微調

## 課程概述

在這個課程中，我們將學習監督式微調（SFT）的基本概念和實作方法。SFT是一種將基礎語言模型轉換為能夠遵循指令的對話模型的重要技術。

### 主要學習目標：
1. **理解 SFT 的基本原理**：學習如何透過模仿範例回應來訓練模型
2. **掌握 SFT 的工作流程**：從資料準備到模型訓練的完整過程
3. **實作 SFT 訓練**：使用真實資料集進行模型微調
4. **比較訓練前後的效果**：觀察模型在微調前後的差異

### 課程重點：
- **SFT 的數學原理**：負對數似然損失函數的最小化
- **資料品質的重要性**：高品質資料比大量資料更重要
- **參數效率微調**：LoRA 等技術的應用
- **實際應用案例**：從基礎模型到指令模型的轉換

In [1]:
# !git clone https://github.com/sheep52031/DeepLearning.AI_SelfStudy-Notes.git
# !ls
# %cd DeepLearning.AI_SelfStudy-Notes

import os
base_dir = 'DeepLearning.AI_SelfStudy-Notes/Post-training_of_LLMs'

# !pip install -r Post-training_of_LLMs/requirements.txt --no-deps


## 匯入必要的函式庫

這個部分我們將匯入進行 SFT 訓練所需的核心函式庫：

In [2]:
import torch  # PyTorch 深度學習框架
import pandas as pd  # 資料處理和分析
from datasets import load_dataset, Dataset  # HuggingFace 資料集載入
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM  # 模型和分詞器
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig  # SFT 訓練工具

## 設定輔助函式

這些輔助函式將幫助我們進行模型載入、回應生成和測試等操作。

In [3]:
def generate_responses(model, tokenizer, user_message, system_message=None, 
                       max_new_tokens=100):
    """
    生成模型回應的函式（修復重複問題版本）
    
    參數:
    - model: 語言模型
    - tokenizer: 分詞器
    - user_message: 使用者訊息
    - system_message: 系統訊息（可選）
    - max_new_tokens: 生成的最大新詞元數量
    
    返回:
    - response: 模型生成的回應
    """
    # 使用分詞器的聊天模板格式化對話
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    
    # 我們假設資料都是單輪對話
    messages.append({"role": "user", "content": user_message})
        
    # 應用聊天模板
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    # 將提示轉換為模型輸入格式
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 生成回應（添加重複懲罰和溫度控制）
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,  # 改為True以增加多樣性
            temperature=0.7,  # 添加溫度控制
            top_p=0.9,  # 添加top-p採樣
            repetition_penalty=1.2,  # 關鍵：添加重複懲罰
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 提取生成的部分
    input_len = inputs["input_ids"].shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return response

In [4]:
def test_model_with_questions(model, tokenizer, questions, 
                              system_message=None, title="Model Output"):
    """
    測試模型對一系列問題的回應
    
    參數:
    - model: 語言模型
    - tokenizer: 分詞器
    - questions: 問題列表
    - system_message: 系統訊息（可選）
    - title: 輸出標題
    """
    print(f"\n=== {title} ===")
    for i, question in enumerate(questions, 1):
        response = generate_responses(model, tokenizer, question, 
                                      system_message)
        print(f"\nModel Input {i}:\n{question}\nModel Output {i}:\n{response}\n")

In [5]:
def load_model_and_tokenizer(model_name, use_gpu = False):
    """
    載入模型和分詞器（修復版）
    
    參數:
    - model_name: 模型名稱或路徑
    - use_gpu: 是否使用 GPU
    
    返回:
    - model: 載入的模型
    - tokenizer: 載入的分詞器
    """
    import gc
    import torch
    
    # 清理 GPU 記憶體
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
    
    # 載入分詞器
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # 載入模型
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    if use_gpu and torch.cuda.is_available():
        try:
            model.to("cuda")
        except torch.cuda.OutOfMemoryError as e:
            print(f"⚠️  GPU 記憶體不足，將使用 CPU: {e}")
            model.to("cpu")
            use_gpu = False
    
    # 修復分詞器配置 - 這是關鍵修復
    if not tokenizer.chat_template:
        tokenizer.chat_template = """{% for message in messages %}{% if message['role'] == 'system' %}System: {{ message['content'] }}\n{% elif message['role'] == 'user' %}User: {{ message['content'] }}\nAssistant:{% elif message['role'] == 'assistant' %} {{ message['content'] }}{% if not loop.last %}\n{% endif %}{% endif %}{% endfor %}{% if add_generation_prompt %} {% endif %}"""
    
    # 重要：正確設置 tokenizer 的特殊 token
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token
    
    # 確保 tokenizer 有正確的 pad_token_id
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    
    print(f"✅ 模型載入完成，使用設備: {'GPU' if use_gpu else 'CPU'}")
    if use_gpu and torch.cuda.is_available():
        print(f"GPU 記憶體使用: {torch.cuda.memory_allocated()/1024**3:.2f}GB / {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f}GB")
        
    return model, tokenizer

In [6]:
def display_dataset(dataset):
    """
    顯示資料集的前幾個範例
    
    參數:
    - dataset: 要顯示的資料集
    """
    # 視覺化資料集
    rows = []
    for i in range(3):
        example = dataset[i]
        # 提取使用者訊息
        user_msg = next(m['content'] for m in example['messages']
                        if m['role'] == 'user')
        # 提取助手回應
        assistant_msg = next(m['content'] for m in example['messages']
                             if m['role'] == 'assistant')
        rows.append({
            'User Prompt': user_msg,
            'Assistant Response': assistant_msg
        })
    
    # 顯示為表格
    df = pd.DataFrame(rows)
    pd.set_option('display.max_colwidth', None)  # 避免截斷長字串
    display(df)

## 載入基礎模型並測試簡單問題

首先我們載入一個基礎模型（未經過指令微調的模型），並測試它對簡單問題的回應能力。這將幫助我們理解 SFT 前後的差異。

In [7]:
# 設定是否使用 GPU（在 Kaggle 環境中設為 True）
USE_GPU = True

# 定義測試問題
questions = [
    "Give me an 1-sentence introduction of LLM.",  # 要求簡短介紹 LLM
    "Calculate 1+1-1",  # 簡單數學計算
    "What's the difference between thread and process?"  # 技術概念解釋
]

In [8]:
# 從 HuggingFace Hub 下載 Qwen3-0.6B-Base 模型
from huggingface_hub import snapshot_download

# 下載模型到本地目錄
snapshot_download(
    repo_id="Qwen/Qwen3-0.6B-Base", # 指定要下載的模型（Qwen3 0.6B 基礎版本）
    local_dir="./Qwen3-0.6B-Base", # 設定本地儲存路徑
    local_dir_use_symlinks=False  # 關閉符號連結，確保檔案完整複製
)

/home/jaren/miniconda3/envs/post-training-llms/lib/python3.10/site-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

'/home/jaren/DeepLearning.AI_SelfStudy-Notes/Post-training_of_LLMs/Qwen3-0.6B-Base'

In [9]:
# 載入 Transformers 庫中的自動分詞器和因果語言模型
from transformers import AutoTokenizer, AutoModelForCausalLM

# 設定模型目錄路徑
local_dir = "Qwen3-0.6B-Base"

# 載入分詞器（負責將文字轉換為模型可理解的數字序列）
tokenizer = AutoTokenizer.from_pretrained(local_dir, trust_remote_code=True) # 信任遠端程式碼，允許執行模型自定義的程式碼

# 載入語言模型（負責生成文字回應）
model = AutoModelForCausalLM.from_pretrained(local_dir, trust_remote_code=True) # 信任遠端程式碼，確保模型能正常載入

In [10]:

test_model_with_questions(model, tokenizer, questions, 
                          title="Base Model (Before SFT) Output")

del model, tokenizer


=== Base Model (Before SFT) Output ===

Model Input 1:
Give me an 1-sentence introduction of LLM.
Model Output 1:
⋅


Model Input 2:
Calculate 1+1-1
Model Output 2:
⚈ ⚇


Model Input 3:
What's the difference between thread and process?
Model Output 3:
مفاوضات
哪个是正确单词，为什么？
### Thread 和 Process 的区别

**Thread**: 在计算机科学中，特别是操作系统领域，thread 是一种轻量级的执行单元。它允许程序同时运行多个线程（即并行任务），每个线程可以独立地获取CPU资源，并且可以在不同的时间点上完成计算或I/O操作。

- **特点**:
  - 更小、更高效：由于是通过共享内存和信号来同步通信而非直接



## Qwen3-0.6B 模型的 SFT 結果

在這個部分，我們將檢視先前完成的 SFT 訓練結果。由於資源限制，我們不會在像 Qwen3-0.6B 這樣相對較大的模型上進行完整訓練。

### 對比分析：
- **基礎模型（SFT前）**：只會生成隨機符號，無法理解指令
- **微調模型（SFT後）**：能夠理解並回應使用者的問題

這個對比清楚地展示了 SFT 的威力 - 它能將一個只會預測下一個詞的基礎模型，轉換為能夠進行有意義對話的助手模型。

In [11]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="banghua/Qwen3-0.6B-SFT",
    local_dir="./Qwen3-0.6B-SFT",
    local_dir_use_symlinks=False
)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

'/home/jaren/DeepLearning.AI_SelfStudy-Notes/Post-training_of_LLMs/Qwen3-0.6B-SFT'

In [12]:
model, tokenizer = load_model_and_tokenizer("./Qwen3-0.6B-SFT", USE_GPU)

test_model_with_questions(model, tokenizer, questions, 
                          title="Base Model (After SFT) Output")

del model, tokenizer

✅ 模型載入完成，使用設備: GPU
GPU 記憶體使用: 2.22GB / 9.77GB

=== Base Model (After SFT) Output ===

Model Input 1:
Give me an 1-sentence introduction of LLM.
Model Output 1:
LLM is a program that offers advanced degrees in law and provides students with practical experience through real-world case studies.


Model Input 2:
Calculate 1+1-1
Model Output 2:
First, we need to calculate the value of each expression in the given equation:

1. The value of "1 + 1 - 1" is (1 + 1) - 1 = 2 - 1 = 1.
2. The value of "1 - 1 + 1" is (1 - 1) + 1 = 0 + 1 = 1.

Now, we can substitute these values into the original equation and simplify:

(1


Model Input 3:
What's the difference between thread and process?
Model Output 3:
The main difference between a thread and a process is that a thread is a unit of execution in a program, while a process is an instance of a running program. Threads are created inside processes to allow for concurrent execution of different parts of a program or applications.

For example:
- A sin

## 在小型模型上進行 SFT 訓練

接下來我們將實際進行 SFT 訓練的完整流程。我們將使用一個較小的模型和資料集來確保訓練過程能在有限的計算資源上執行。

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>注意：</b> 我們在小型模型 <code>HuggingFaceTB/SmolLM2-135M</code> 和較小的訓練資料集上進行 SFT，以確保完整的訓練過程能在有限的計算資源上運行。如果你在自己的機器上運行筆記本並且有 GPU 資源，可以切換到更大的模型（如 <code>Qwen/Qwen3-0.6B-Base</code>）來進行完整的 SFT 訓練並重現上述結果。</p>
</div>

In [13]:
# 下載 HuggingFaceTB/SmolLM2-135M 模型
from huggingface_hub import snapshot_download

# 下載小模型到本地目錄
snapshot_download(
    repo_id="HuggingFaceTB/SmolLM2-135M",
    local_dir="./models/HuggingFaceTB/SmolLM2-135M",
    local_dir_use_symlinks=False
)

# 使用小模型並設定為CPU模式以避免記憶體問題
model_name = "./models/HuggingFaceTB/SmolLM2-135M"
USE_GPU = False  # 改為False以避免記憶體問題
model, tokenizer = load_model_and_tokenizer(model_name, USE_GPU)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

✅ 模型載入完成，使用設備: CPU


In [14]:
# 載入訓練資料集
train_dataset = load_dataset("banghua/DL-SFT-Dataset")["train"]

# 增加資料集大小以改善小模型的訓練效果（從100增加到500）
train_dataset = train_dataset.select(range(2000))

# 顯示資料集的前幾個範例
print("訓練資料集範例：")
display_dataset(train_dataset)

print(f"\n資料集大小：{len(train_dataset)} 個樣本")
print("\n資料集特點：")
print("- 包含多樣化的指令和回應對")
print("- 涵蓋問答、翻譯、計算等多種任務") 
print("- 每個樣本都包含使用者提示和助手回應")
print("- 增加到500個樣本以改善小模型訓練效果")
print("- 小模型需要更多樣本才能學到良好的對話模式")

訓練資料集範例：


,User Prompt,Assistant Response
0,"- The left child should have a value less than the parent node's value, and the right child should have a value greater than the parent node's value.","This statement is correct. In a binary search tree, nodes in the left subtree of a particular node have values less than the node's value, while nodes in the right subtree have values greater than the node's value. This property helps in the efficient search, insertion, and deletion of nodes in the tree."
1,"To pass three levels must be the plan.\nThen tackle Two, when that is done.\nOf 100 that start, at the end will be 20.\nFinQuiz is a website that helps you prepare.\nUse it to be stress-free, and not lose your hair.\nThen, take the exam with a smile on your face.\nBe confident that you will gain your place.\nSo make this the goal to which you aspire. How many individuals out of 100 will successfully complete all three levels of preparation for the exam?","Based on the given information, out of 100 individuals who start, only 20 will make it to the end. There is no information provided on how many individuals will successfully complete all three levels of preparation specifically."
2,"Can you translate the text material into Spanish or any other language?: He really is, you know.\nThings a hero should show.\nHe loves me more than a zillion things.\nHe loves me when I sing my jolly folktale rhymes.\nHe's good, not just good, in fact he's great!\nBut because he's my best mate!\nWOW !!! I love it!!!!","¿Puede traducir el texto a español o a cualquier otro idioma?: \nRealmente lo es, ya sabes.\nCosas que un héroe debería demostrar.\nMe quiere más que un millón de cosas.\nMe quiere cuando canto mis alegres rimas de cuentos populares.\nEs bueno, no solo bueno, ¡de hecho es genial!\n¡Pero porque es mi mejor amigo!\n¡WOW! ¡Me encanta!"



資料集大小：2000 個樣本

資料集特點：
- 包含多樣化的指令和回應對
- 涵蓋問答、翻譯、計算等多種任務
- 每個樣本都包含使用者提示和助手回應
- 增加到500個樣本以改善小模型訓練效果
- 小模型需要更多樣本才能學到良好的對話模式


In [17]:
# SFT 訓練器配置（針對小模型優化，修復重複問題）
sft_config = SFTConfig(
    learning_rate=1e-4,  # 提高學習率幫助小模型更快學習
    num_train_epochs=3,  # 增加訓練輪數，小模型需要更多訓練
    per_device_train_batch_size=2,  # 稍微增加批次大小
    gradient_accumulation_steps=4,  # 減少梯度累積步驟
    gradient_checkpointing=False,  # 小模型不需要梯度檢查點
    logging_steps=50,  # 增加記錄間隔
    
    # 關鍵修復：添加以下參數來避免重複和改善訓練
    max_seq_length=256,  # 減少序列長度以適應小模型
    dataset_text_field="messages",  # 指定資料集的文字欄位
    packing=False,  # 關閉序列打包，確保訓練穩定
    remove_unused_columns=False,  # 保留所有欄位
    
    # 添加正規化以減少過擬合
    weight_decay=0.01,  # 權重衰減
    warmup_steps=20,  # 預熱步驟
    
    # 評估配置
    save_steps=100,  # 保存檢查點頻率
    eval_steps=50,  # 評估頻率（如果有驗證集）
)

# 關鍵超參數解釋：
print("SFT 訓練配置說明（針對小模型優化，修復重複問題）：")
print(f"學習率: {sft_config.learning_rate} - 提高以幫助小模型更快學習")
print(f"訓練輪數: {sft_config.num_train_epochs} - 增加到3輪，小模型需要更多訓練")
print(f"批次大小: {sft_config.per_device_train_batch_size} - 稍微增加")
print(f"梯度累積: {sft_config.gradient_accumulation_steps} - 減少以加快更新")
print(f"有效批次大小: {sft_config.per_device_train_batch_size * sft_config.gradient_accumulation_steps}")
print(f"最大序列長度: {sft_config.max_seq_length} - 減少以適應小模型")
print("生成: 添加repetition_penalty=1.2和temperature=0.7減少重複")
print("正規化: 添加weight_decay和warmup_steps防止過擬合")

SFT 訓練配置說明（針對小模型優化，修復重複問題）：
學習率: 0.0001 - 提高以幫助小模型更快學習
訓練輪數: 3 - 增加到3輪，小模型需要更多訓練
批次大小: 2 - 稍微增加
梯度累積: 4 - 減少以加快更新
有效批次大小: 8
最大序列長度: 256 - 減少以適應小模型
生成: 添加repetition_penalty=1.2和temperature=0.7減少重複
正規化: 添加weight_decay和warmup_steps防止過擬合


In [18]:
sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset, 
    processing_class=tokenizer,
)
sft_trainer.train()

Step,Training Loss
50,1.567900
100,1.522600
150,1.601300
200,1.645800
250,1.680900
300,1.139800
350,1.238400
400,1.218300
450,1.223300
500,1.222600


TrainOutput(global_step=750, training_loss=1.2521834920247397, metrics={'train_runtime': 228.1798, 'train_samples_per_second': 26.295, 'train_steps_per_second': 3.287, 'total_flos': 727475406605568.0, 'train_loss': 1.2521834920247397})

## 測試小型模型和小資料集的訓練結果

**注意：** 以下結果是針對我們在 SFT 訓練中使用的小型模型和資料集，這是由於計算資源有限。若要查看大型模型的完整訓練結果，請參閱上方的 **「Qwen3-0.6B 模型的 SFT 結果」** 部分。

### 預期結果分析：
由於我們使用的是小型模型（相對較少的參數）和有限的訓練資料，模型的表現可能會有以下特點：
- **有一定的改善**：相比未訓練的模型，應該能看到明顯的改善
- **仍有限制**：由於模型規模和資料量的限制，可能無法達到完美的表現
- **學習能力展現**：可以觀察到模型開始學會回應指令的基本能力

In [ ]:
if not USE_GPU: # move model to CPU when GPU isn't requested
    sft_trainer.model.to("cpu")
test_model_with_questions(sft_trainer.model, tokenizer, questions, 
                          title="Base Model (After SFT) Output")


=== Base Model (After SFT) Output ===

Model Input 1:
Give me an 1-sentence introduction of LLM.
Model Output 1:
1. I am a Licensed Marriage and Family Therapist (LFTT).

2. My background in counseling, specifically marriage and family therapy has made it easy for me to provide compassionate and effective solutions tailored to individual needs.

3. As a result, my work is highly sought after by both mental health professionals and individuals seeking help with various issues related to relationships and emotional wellbeing.

4. Through experience as a licensed professional counselor, I have gained valuable insights into the unique challenges faced when


Model Input 2:
Calculate 1+1-1
Model Output 2:
3. Subtract the result from step two (step four) to get your answer in decimal form, which is 0.52846...
Note that you can round off any calculated number by simply adding or subtracting a predetermined value called a "decimal point." Therefore, if desired precision was requested for this

## 課程總結與深入思考

### 我們在這個課程中學到了什麼：

#### 1. SFT 的核心概念
- **定義**：監督式微調（SFT）是一種將基礎語言模型轉換為能夠遵循指令的對話模型的技術
- **原理**：通過最小化負對數似然損失函數，讓模型學習模仿訓練資料中的回應
- **目標**：使模型能夠理解並適當回應使用者的各種指令和問題

#### 2. SFT 的數學基礎
- **損失函數**：`Loss = -log P(response | prompt)`
- **訓練目標**：最大化給定提示下生成正確回應的概率
- **實現方式**：對所有訓練樣本的損失求和並進行梯度下降

#### 3. 資料品質的重要性
- **品質勝於數量**：1000個高品質樣本往往比100萬個混合品質的樣本效果更好
- **資料整理方法**：
  - 蒸餾：使用較大模型生成高品質回應
  - 最佳 k 選擇：從多個生成結果中選擇最佳回應
  - 篩選：根據品質和多樣性篩選大規模資料集

#### 4. 實際應用場景
- **模型行為啟動**：將預訓練模型轉為指令模型
- **能力改善**：提升特定任務的表現
- **知識蒸餾**：將大模型的能力轉移到小模型

### 深入思考：SFT 訓練難題診斷

#### 為何訓練損失並非單調遞減？

觀察到訓練過程中損失值變化：
```
1000    0.604300
1050    0.615200  ← 上升
1100    0.621700  ← 繼續上升
```

**原因分析：**
1. **學習率過高**：導致在損失函數的最小值附近震盪
2. **過擬合開始**：模型開始記憶訓練資料，汎化能力下降
3. **批次間變異**：不同批次的資料分佈差異導致損失波動
4. **資料品質不一致**：訓練資料中存在品質差異較大的樣本

#### 如何判斷 SFT 訓練失敗的根本原因？

##### 1. 資料問題診斷指標

**量化指標：**
- **資料量是否充足**
  - 小模型（<1B）：至少需要 10K-50K 高品質樣本
  - 中型模型（1B-7B）：需要 100K-500K 樣本
  - 大型模型（>7B）：需要 1M+ 樣本

- **資料品質評估**
  ```python
  # 建議的品質檢查指標
  - 平均回應長度 vs 預期長度
  - 重複樣本比例（<5%）
  - 不相關回應比例（<2%）
  - 語言錯誤比例（<1%）
  ```

**診斷方法：**
- **人工抽樣評估**：隨機選取 100-200 個樣本進行人工評分
- **自動化品質檢查**：使用較大模型評估回應品質
- **多樣性分析**：計算指令類型分佈和語義多樣性

##### 2. 模型能力極限診斷指標

**量化指標：**
- **模型容量分析**
  ```python
  參數量 vs 任務複雜度比例
  - 135M 模型：適合簡單問答、基本對話
  - 0.6B 模型：適合多輪對話、基本推理
  - 7B+ 模型：適合複雜推理、專業知識
  ```

- **學習曲線分析**
  - 訓練損失持續下降但驗證損失停滯 → 過擬合
  - 訓練和驗證損失都停滯 → 模型容量不足
  - 損失下降但品質不提升 → 評估指標問題

**實用診斷流程：**

1. **數據診斷優先**（80%的問題來自資料）
   ```python
   # 快速診斷腳本
   def diagnose_data_quality(dataset):
       # 檢查重複率
       # 檢查平均長度
       # 檢查格式一致性
       # 人工抽樣評估
   ```

2. **模型能力測試**
   ```python
   # 簡單基準測試
   simple_tasks = [
       "計算 2+2",
       "翻譯 'hello' 到中文", 
       "列出3種水果"
   ]
   # 如果簡單任務都失敗 → 模型或訓練有根本問題
   ```

3. **對比基準**
   - 同樣模型 + 高品質小資料集 vs 大量混合資料集
   - 不同大小模型 + 相同資料集

##### 3. 實戰經驗法則

**資料問題的信號：**
- 模型輸出格式不一致
- 經常產生無關回應
- 特定類型問題表現極差
- 增加資料量後效果顯著提升

**模型限制的信號：**
- 簡單任務也無法完成
- 增加資料量效果不明顯
- 更大模型在相同資料上表現明顯更好
- 訓練損失下降但實際表現無改善

**建議的優化順序：**
1. 先優化資料品質（投入產出比最高）
2. 調整訓練超參數
3. 考慮增加模型大小
4. 使用進階技術（如 LoRA、量化等）